In [1]:
%pip install "dask[complete]"
%pip install dask_ml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import warnings
warnings.filterwarnings('ignore')

In [4]:
fl_dask = dd.read_csv("../flights/flights.csv",dtype={"arr_mins_of_delay": "object", "dep_mins_of_delay": "object"})
mt_dask = dd.read_csv("../metar/metars.csv")
mad_dask = dd.read_csv("../eurocontrol_MAD_data/MAD_data.csv")

In [5]:
fl_dask.head()

,flight_id,departure_date_time,cod_flight_IATA,cod_flight_ICAO,day,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,...,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta
0,1,2023-10-31 23:59:00,IB6841,IBE6841,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,...,late,33,Buenos Aires,EZE,SAEZ,08:29 UTC-03,on time,0,12h,33
1,2,2023-10-31 23:59:00,IB6011,IBE6011,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,...,late,25,Montevideo,MVD,SUMU,08:48 UTC-03,early,-6,12h,31
2,3,2023-10-31 23:59:00,IB6833,IBE6833,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,...,late,35,Santiago,SCL,SCEL,09:13 UTC-03,early,-6,13h,41
3,4,2023-10-31 23:55:00,IB6827,IBE6827,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,...,late,32,Sao Paulo,GRU,SBGR,06:49 UTC-03,early,-15,10h,47
4,5,2023-10-31 23:55:00,IB6589,IBE6589,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,...,late,24,Bogota,BOG,SKBO,03:52 UTC-05,early,-27,10h,51


In [6]:
mt_dask.head()

,Metar_id,Date_time,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,1,2023-10-31 23:30:00,2023-10-31,23:30,Fair,8,3,0,93,1017
1,2,2023-10-31 23:00:00,2023-10-31,23:00,Fair,8,1,0,87,1017
2,3,2023-10-31 22:30:00,2023-10-31,22:30,Fair,8,0,0,93,1017
3,4,2023-10-31 22:00:00,2023-10-31,22:00,Clear,8,0,0,93,1017
4,5,2023-10-31 21:30:00,2023-10-31,21:30,Clear,8,0,0,93,1017


In [7]:
fl_dask = fl_dask.rename(columns={'departure_date_time': 'Date_time'})

In [8]:
mt_dask['Date_time'] = dd.to_datetime(mt_dask['Date_time'])
fl_dask['Date_time'] = dd.to_datetime(fl_dask['Date_time'])


In [9]:
fl_dask = fl_dask.sort_values('Date_time')
mt_dask = mt_dask.sort_values('Date_time')


In [10]:
%%time
fl_dask = fl_dask.sort_values('Date_time')
mtr_dask = mt_dask.sort_values('Date_time')

# Realiza el merge asof para encontrar el parte meteorológico más cercano en el tiempo
fl_dask = dd.merge_asof(fl_dask, mt_dask, on='Date_time', direction='backward')


CPU times: total: 578 ms
Wall time: 820 ms


In [11]:
pd.set_option('display.max_columns', None)

In [12]:
fl_dask.head()

,Date_time,flight_id,cod_flight_IATA,cod_flight_ICAO,day,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,Scheduled_dep,depart_time,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,2022-11-01 00:25:00,176596,AV47,AVA047,2022-11-01,Tuesday,Landed,Avianca,AV,AVA,00:25,00:38,late,13,Bogota,BOG,SKBO,04:14 UTC-05,early,-35,-,48,17722,2022-11-01,00:00,Mostly Cloudy,14,2,0,94,1022
1,2022-11-01 01:45:00,176595,UX39,AEA039,2022-11-01,Tuesday,Landed,Air Europa,UX,AEA,01:45,02:02,late,17,Quito,UIO,SEQM,06:36 UTC-05,late,11,11h,6,17719,2022-11-01,01:30,Mostly Cloudy,14,2,0,94,1023
2,2022-11-01 06:00:00,176594,KL1698,KLM22A,2022-11-01,Tuesday,Landed,KLM Royal Dutch Airlines,KL,KLM,06:00,06:08,late,8,Amsterdam,AMS,EHAM,08:11 CET,early,-28,2h,36,17710,2022-11-01,06:00,Mostly Cloudy,14,1,0,94,1023
3,2022-11-01 06:10:00,176593,FR2084,RYR69HY,2022-11-01,Tuesday,Landed,Ryanair,FR,RYR,06:10,06:34,late,24,Ibiza,IBZ,LEIB,07:38 CET,late,8,1h,16,17710,2022-11-01,06:00,Mostly Cloudy,14,1,0,94,1023
4,2022-11-01 06:10:00,176592,FR5997,RYR93XH,2022-11-01,Tuesday,Landed,Ryanair,FR,RYR,06:10,06:32,late,22,London,STN,EGSS,07:31 GMT,early,-8,2h,30,17710,2022-11-01,06:00,Mostly Cloudy,14,1,0,94,1023


In [13]:
fl_dask = fl_dask.sort_values('Date_time', ascending=False)
fl_dask.head()

,Date_time,flight_id,cod_flight_IATA,cod_flight_ICAO,day,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,Scheduled_dep,depart_time,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
176595,2023-10-31 23:59:00,1,IB6841,IBE6841,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:32,late,33,Buenos Aires,EZE,SAEZ,08:29 UTC-03,on time,0,12h,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017
176594,2023-10-31 23:59:00,3,IB6833,IBE6833,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:34,late,35,Santiago,SCL,SCEL,09:13 UTC-03,early,-6,13h,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017
176593,2023-10-31 23:59:00,2,IB6011,IBE6011,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:24,late,25,Montevideo,MVD,SUMU,08:48 UTC-03,early,-6,12h,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017
176592,2023-10-31 23:55:00,4,IB6827,IBE6827,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:55,00:27,late,32,Sao Paulo,GRU,SBGR,06:49 UTC-03,early,-15,10h,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017
176591,2023-10-31 23:55:00,7,UX41,AEA041,2023-10-31,Tuesday,Landed,Air Europa,UX,AEA,23:55,00:11,late,16,Buenos Aires,EZE,SAEZ,08:14 UTC-03,early,-35,12h,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017


In [14]:
fl_dask = fl_dask.rename(columns={'day': 'Date'})

In [15]:
fl_dask['Date'] = dd.to_datetime(fl_dask['Date'])
mad_dask['Date'] = dd.to_datetime(mad_dask['Date'])

# Ordenar ambos DataFrames por la columna de fecha
fl_dask = fl_dask.sort_values('Date')
mad_dask = mad_dask.sort_values('Date')

# Realizar el merge asof para encontrar el registro de tráfico más cercano en el tiempo
fl_dask = dd.merge_asof(fl_dask, mad_dask, on='Date', direction='backward')

In [16]:
reorder = ['Date_time','flight_id','Metar_id','mad_id', 'cod_flight_IATA', 'cod_flight_ICAO', 'Date',
       'week_day', 'status', 'airliner', 'cod_airliner_IATA',
       'cod_airliner_ICAO', 'Scheduled_dep', 'depart_time', 'dep_situation',
       'dep_mins_of_delay', 'city', 'cod_airport_IATA', 'cod_airport_ICAO',
       'arrival', 'arr_situation', 'arr_mins_of_delay', 'duration', 'resta']
fl_dask = fl_dask[reorder]

In [17]:
fl_dask.head()

,Date_time,flight_id,Metar_id,mad_id,cod_flight_IATA,cod_flight_ICAO,Date,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,Scheduled_dep,depart_time,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta
0,2022-11-01 00:25:00,176596,17722,365,AV47,AVA047,2022-11-01,Tuesday,Landed,Avianca,AV,AVA,00:25,00:38,late,13,Bogota,BOG,SKBO,04:14 UTC-05,early,-35,-,48
1,2022-11-01 16:20:00,176285,17690,365,IB8700,ANE88YB,2022-11-01,Tuesday,Landed,Iberia,IB,IBE,16:20,16:48,late,28,Bordeaux,BOD,LFBD,17:41 CET,early,-3,1h,31
2,2022-11-01 16:20:00,176286,17690,365,AF1601,AFR75YB,2022-11-01,Tuesday,Landed,Air France,AF,AFR,16:20,16:39,late,19,Paris,CDG,LFPG,18:13 CET,early,-16,2h,35
3,2022-11-01 16:15:00,176291,17690,365,IB3244,IBE32LV,2022-11-01,Tuesday,Landed,Iberia,IB,IBE,16:15,16:42,late,27,Venice,VCE,LIPZ,18:33 CET,early,-11,2h,38
4,2022-11-01 16:15:00,176292,17690,365,UX89,AEA089,2022-11-01,Tuesday,Landed,Air Europa,UX,AEA,16:15,16:41,late,26,Santo Domingo,SDQ,MDSD,20:07 AST,late,2,8h,24


In [18]:
fl_dask = fl_dask.sort_values('Date_time', ascending=False).reset_index(drop = True)

In [19]:
fl_dask = fl_dask.rename(columns ={"Date_time": "departure_date_time","Metar_id": "metar_id", "Date": "day"})

In [23]:
fl_dask.head()

,departure_date_time,flight_id,metar_id,mad_id,cod_flight_IATA,cod_flight_ICAO,day,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,Scheduled_dep,depart_time,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta
0,2023-10-31 23:59:00,1,1,1,IB6841,IBE6841,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:32,late,33,Buenos Aires,EZE,SAEZ,08:29 UTC-03,on time,0,12h,33
1,2023-10-31 23:59:00,3,1,1,IB6833,IBE6833,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:34,late,35,Santiago,SCL,SCEL,09:13 UTC-03,early,-6,13h,41
2,2023-10-31 23:59:00,2,1,1,IB6011,IBE6011,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:24,late,25,Montevideo,MVD,SUMU,08:48 UTC-03,early,-6,12h,31
3,2023-10-31 23:55:00,4,1,1,IB6827,IBE6827,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:55,00:27,late,32,Sao Paulo,GRU,SBGR,06:49 UTC-03,early,-15,10h,47
4,2023-10-31 23:55:00,7,1,1,UX41,AEA041,2023-10-31,Tuesday,Landed,Air Europa,UX,AEA,23:55,00:11,late,16,Buenos Aires,EZE,SAEZ,08:14 UTC-03,early,-35,12h,51


In [21]:
fl2 = fl_dask.compute()

In [27]:
fl2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176596 entries, 0 to 176595
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   departure_date_time  176596 non-null  datetime64[ns]
 1   flight_id            176596 non-null  int64         
 2   metar_id             176596 non-null  int64         
 3   mad_id               176596 non-null  int64         
 4   cod_flight_IATA      176596 non-null  object        
 5   cod_flight_ICAO      173331 non-null  object        
 6   day                  176596 non-null  datetime64[ns]
 7   week_day             176596 non-null  object        
 8   status               176596 non-null  object        
 9   airliner             176596 non-null  object        
 10  cod_airliner_IATA    176596 non-null  object        
 11  cod_airliner_ICAO    176596 non-null  object        
 12  Scheduled_dep        176596 non-null  object        
 13  depart_time   

In [22]:
fl2.to_csv("../flights/flights_with_ids.csv", index = False)

In [28]:
fl2.to_parquet("../flights/flights_with_ids.gz", compression='gzip', index = False)

In [30]:
fl2.to_json("../flights/flights_with_ids.json")